In [1]:
import torch
import pandas as pd
import numpy as np
import sys 
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler, StandardScaler

sys.path.append("./")
from prediction.api import *
from prediction.data_wrapper import get_loaders
from pickle import dump

In [2]:
args.data = './data/processed/processed_cleaned_df_2.csv'
args.input_size = [33]
args.hidden_size = 100
args.number_of_layers = 1
args.output_size = [1]
args.test_portion = 0.2 
args.train_window = 12
args.batch_size = 64
args.learning_rate = 0.00001
args.weight_decay = 0.0001
args.gpu = 0 
args.epochs = 10
args.log_freq = 30
args.debug = False
args.seed = 1
args.samples = 20
args.dropout = 0.3

In [3]:
df = pd.read_csv(args.data).iloc[:,1:]
label_index = df.columns.get_loc('0411')
print(df.head())

      GPs  month      07    0304   0411  050103  131002   010305       01  \
0  A81001      1  7271.0  4071.0  758.0  1476.0   185.0  13032.0  69503.0   
1  A81001      2  8527.0  4071.0  758.0  1427.0   191.0  12740.0  78753.0   
2  A81001      3  6402.0  3928.0  758.0  1278.0   198.0  11574.0  65473.0   
3  A81001      4  7295.0  4836.0  758.0  1537.0   231.0  13019.0  74436.0   
4  A81001      5  6671.0  5153.0  758.0  1436.0   280.0  11824.0  65084.0   

       05  ...  030401       13           12     0106     1302       10  \
0  9767.0  ...  4310.0  59472.0   886.000000  19797.0  49930.0  15657.0   
1  9767.0  ...  4064.0  59472.0   994.666667  24304.0  49930.0  12903.0   
2  9767.0  ...  3928.0  59472.0  1103.333333  20416.0  44955.0  13236.0   
3  9767.0  ...  4836.0  58722.0  1212.000000  24713.0  45005.0  14082.0   
4  9767.0  ...  5152.0  57972.0   817.000000  20309.0  42955.0  12972.0   

    130201  050108    0603   010604  
0  41880.0   998.0  4034.0  10939.0  
1  48830.0

In [4]:
data = df.to_numpy()
gps = data[:,0]
features = data[:,1:]

In [5]:
print(features.shape, gps.shape)

(394458, 33) (394458,)


In [6]:
total_size = len(features)
test_size = int(args.test_portion * total_size)

train_data = features[test_size:]
test_data = features[:test_size]

train_gps = gps[test_size:]
test_gps = gps[:test_size]

train_unique_gps = np.unique(train_gps)
test_unique_gps = np.unique(test_gps)

scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized = scaler.fit_transform(train_data)
test_data_normalized = scaler.transform(test_data)

print(np.min(train_data_normalized, axis=0))
print(np.max(train_data_normalized, axis=0))
dump(scaler, open("model/scaler.pck", "wb"))
print(np.min(test_data_normalized, axis=0))
print(np.max(test_data_normalized, axis=0))

train_data_normalized = torch.FloatTensor(train_data_normalized)
test_data_normalized = torch.FloatTensor(test_data_normalized)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[-1.         -0.9999891  -1.         -0.99935557 -0.99933241 -1.
 -0.99983128 -0.99992944 -0.99469687 -1.         -1.         -1.0000085
 -1.         -0.99958916 -1.         -0.99982458 -1.         -0.99996968
 -0.99915395 -0.9990583  -0.99967836 -1.         -1.         -0.99954958
 -0.99979465 -1.         -0.99987044 -0.99987595 -0.99957195 -0.99987359
 -0.99969052 -0.99952227 -0.9998063 ]
[ 1.          0.67267932  0.32723297  0.54857419  0.75933241  1.54167338
  0.39223936  0.66482492  0.78379221  0.3211487   0.7274388   0.26488443
  0.68517796  0.86285371  1.88033886  0.44255542  0.26276169  0.21406535
  0.33020764  0.21016616  0.4747188   0.71898384 -0.10585794  0.32837351
  0.3208946   0.16316144  0.35057432  0.34705103  0.98851281  0.30666968
 -0.

In [7]:
def create_inout_sequences(input_data,_gps,_unique_gps,_label_index, tw):
    X = []
    Y = []
    counter = 0
    for i in tqdm(range(len(input_data)-tw-1)):
        seq = input_data[i:i+tw]
        label = input_data[i+tw:i+tw+1,_label_index]
        if counter == 57:
            counter = 0
            continue
        seq = np.array(seq, dtype=np.float32)
        label = np.array(label, dtype = np.float32)
        X.append(torch.from_numpy(seq).float())
        Y.append(torch.from_numpy(label).float())
        counter+=1

    return X, Y

X_train, Y_train = create_inout_sequences(train_data_normalized, train_gps, train_unique_gps, label_index, args.train_window)
X_test, Y_test = create_inout_sequences(test_data_normalized, test_gps, test_unique_gps, label_index, args.train_window)
print(np.unique(Y_train))
print(np.unique(Y_test))

100%|██████████| 78878/78878 [00:01<00:00, 48558.37it/s]


[-1.        -0.9997775 -0.9996662 ...  0.9189986  0.9581641  1.       ]
[-0.9993324  -0.9986648  -0.9984423  ...  0.55182195  0.65785813
  0.7593324 ]


In [8]:
X_train = torch.stack(X_train)
X_test = torch.stack(X_test)
Y_train = torch.stack(Y_train)
Y_test = torch.stack(Y_test)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

torch.Size([310114, 12, 33]) torch.Size([310114, 1])
torch.Size([77519, 12, 33]) torch.Size([77519, 1])


In [9]:
print(X_train[0,11], Y_train[0])


tensor([-1.0000, -0.7952, -0.8208, -0.9017, -0.7634, -0.8304, -0.6978, -0.4449,
        -0.7710, -0.7906, -0.8780, -0.7974, -0.5802, -0.7848, -0.9280, -0.6143,
        -0.9103, -0.7838, -0.8123, -0.7189, -0.7158, -0.9288, -0.9154, -0.8206,
        -0.7590, -0.8540, -0.6602, -0.7791, -0.6972, -0.7762, -0.9508, -0.8219,
        -0.6973]) tensor([-0.7943])


In [10]:
def get_loaders(X, Y):
    dataset = torch.utils.data.TensorDataset(X,Y)
    return torch.utils.data.DataLoader(dataset,batch_size = args.batch_size,shuffle=False)

In [11]:
train_loader = get_loaders(X_train, Y_train)
test_loader = get_loaders(X_test, Y_test)

In [12]:
model, scaler = get_model(train_loader, test_loader)

03/01 06:16:30 AM | # START #
03/01 06:16:30 AM | Args = Namespace(batch_size=64, data='./data/processed/processed_cleaned_df_2.csv', debug=False, dropout=0.3, epochs=10, f='/home/killko/.local/share/jupyter/runtime/kernel-68e2c5a1-3542-41f4-b3c7-e1f110eda873.json', gpu=0, hidden_size=100, input_size=[33], layers=8, learning_rate=1e-05, log_freq=30, model_path='./model/drugs-EXP-20200301-061618/drugs-EXP-20200301-061618.model', num_of_layers=1, number_of_layers=1, output_size=[1], samples=20, save='./model/drugs-EXP-20200301-061618', seed=1, test_portion=0.2, train_window=12, weight_decay=0.0001)
INFO:net:Args = Namespace(batch_size=64, data='./data/processed/processed_cleaned_df_2.csv', debug=False, dropout=0.3, epochs=10, f='/home/killko/.local/share/jupyter/runtime/kernel-68e2c5a1-3542-41f4-b3c7-e1f110eda873.json', gpu=0, hidden_size=100, input_size=[33], layers=8, learning_rate=1e-05, log_freq=30, model_path='./model/drugs-EXP-20200301-061618/drugs-EXP-20200301-061618.model', num_o

INFO:net:#### Train: [  1/10] Step 1350/4845 MSE 2.1154+-0.00000
03/01 06:16:35 AM | #### Train: [  1/10] Step 1380/4845 MSE 2.0734+-0.00000
INFO:net:#### Train: [  1/10] Step 1380/4845 MSE 2.0734+-0.00000
03/01 06:16:35 AM | #### Train: [  1/10] Step 1410/4845 MSE 2.0343+-0.00000
INFO:net:#### Train: [  1/10] Step 1410/4845 MSE 2.0343+-0.00000
03/01 06:16:35 AM | #### Train: [  1/10] Step 1440/4845 MSE 1.9946+-0.00000
INFO:net:#### Train: [  1/10] Step 1440/4845 MSE 1.9946+-0.00000
03/01 06:16:35 AM | #### Train: [  1/10] Step 1470/4845 MSE 1.9603+-0.00000
INFO:net:#### Train: [  1/10] Step 1470/4845 MSE 1.9603+-0.00000
03/01 06:16:35 AM | #### Train: [  1/10] Step 1500/4845 MSE 1.9280+-0.00000
INFO:net:#### Train: [  1/10] Step 1500/4845 MSE 1.9280+-0.00000
03/01 06:16:35 AM | #### Train: [  1/10] Step 1530/4845 MSE 1.8969+-0.00000
INFO:net:#### Train: [  1/10] Step 1530/4845 MSE 1.8969+-0.00000
03/01 06:16:35 AM | #### Train: [  1/10] Step 1560/4845 MSE 1.8637+-0.00000
INFO:net:####

03/01 06:16:39 AM | #### Train: [  1/10] Step 3120/4845 MSE 1.1073+-0.00000
INFO:net:#### Train: [  1/10] Step 3120/4845 MSE 1.1073+-0.00000
03/01 06:16:39 AM | #### Train: [  1/10] Step 3150/4845 MSE 1.0998+-0.00000
INFO:net:#### Train: [  1/10] Step 3150/4845 MSE 1.0998+-0.00000
03/01 06:16:39 AM | #### Train: [  1/10] Step 3180/4845 MSE 1.0924+-0.00000
INFO:net:#### Train: [  1/10] Step 3180/4845 MSE 1.0924+-0.00000
03/01 06:16:39 AM | #### Train: [  1/10] Step 3210/4845 MSE 1.0878+-0.00000
INFO:net:#### Train: [  1/10] Step 3210/4845 MSE 1.0878+-0.00000
03/01 06:16:39 AM | #### Train: [  1/10] Step 3240/4845 MSE 1.0800+-0.00000
INFO:net:#### Train: [  1/10] Step 3240/4845 MSE 1.0800+-0.00000
03/01 06:16:39 AM | #### Train: [  1/10] Step 3270/4845 MSE 1.0736+-0.00000
INFO:net:#### Train: [  1/10] Step 3270/4845 MSE 1.0736+-0.00000
03/01 06:16:39 AM | #### Train: [  1/10] Step 3300/4845 MSE 1.0664+-0.00000
INFO:net:#### Train: [  1/10] Step 3300/4845 MSE 1.0664+-0.00000
03/01 06:16:3

INFO:net:#### Train: [  1/10] Step 4845/4845 MSE 0.8137+-0.00000
03/01 06:16:43 AM | _train_step_Train: [  1/10] Final MSE 0.8137+-0.00000 Time 11.30
INFO:net:_train_step_Train: [  1/10] Final MSE 0.8137+-0.00000 Time 11.30
03/01 06:16:43 AM | #### Test: [  1/10] Step 030/1211 MSE 1.3565+-0.03822
INFO:net:#### Test: [  1/10] Step 030/1211 MSE 1.3565+-0.03822
03/01 06:16:43 AM | #### Test: [  1/10] Step 060/1211 MSE 0.9455+-0.03229
INFO:net:#### Test: [  1/10] Step 060/1211 MSE 0.9455+-0.03229
03/01 06:16:44 AM | #### Test: [  1/10] Step 090/1211 MSE 1.1037+-0.03490
INFO:net:#### Test: [  1/10] Step 090/1211 MSE 1.1037+-0.03490
03/01 06:16:44 AM | #### Test: [  1/10] Step 120/1211 MSE 0.8960+-0.03083
INFO:net:#### Test: [  1/10] Step 120/1211 MSE 0.8960+-0.03083
03/01 06:16:45 AM | #### Test: [  1/10] Step 150/1211 MSE 1.0490+-0.03190
INFO:net:#### Test: [  1/10] Step 150/1211 MSE 1.0490+-0.03190
03/01 06:16:45 AM | #### Test: [  1/10] Step 180/1211 MSE 1.0270+-0.03055
INFO:net:#### Tes

INFO:net:#### Train: [  2/10] Step 450/4845 MSE 0.3610+-0.00000
03/01 06:17:00 AM | #### Train: [  2/10] Step 480/4845 MSE 0.3610+-0.00000
INFO:net:#### Train: [  2/10] Step 480/4845 MSE 0.3610+-0.00000
03/01 06:17:00 AM | #### Train: [  2/10] Step 510/4845 MSE 0.3473+-0.00000
INFO:net:#### Train: [  2/10] Step 510/4845 MSE 0.3473+-0.00000
03/01 06:17:00 AM | #### Train: [  2/10] Step 540/4845 MSE 0.3351+-0.00000
INFO:net:#### Train: [  2/10] Step 540/4845 MSE 0.3351+-0.00000
03/01 06:17:00 AM | #### Train: [  2/10] Step 570/4845 MSE 0.3272+-0.00000
INFO:net:#### Train: [  2/10] Step 570/4845 MSE 0.3272+-0.00000
03/01 06:17:01 AM | #### Train: [  2/10] Step 600/4845 MSE 0.3171+-0.00000
INFO:net:#### Train: [  2/10] Step 600/4845 MSE 0.3171+-0.00000
03/01 06:17:01 AM | #### Train: [  2/10] Step 630/4845 MSE 0.3067+-0.00000
INFO:net:#### Train: [  2/10] Step 630/4845 MSE 0.3067+-0.00000
03/01 06:17:01 AM | #### Train: [  2/10] Step 660/4845 MSE 0.2971+-0.00000
INFO:net:#### Train: [  2/1

...

INFO:net:#### Train: [ 10/10] Step 390/4845 MSE 0.2826+-0.00000
03/01 06:20:37 AM | #### Train: [ 10/10] Step 420/4845 MSE 0.2911+-0.00000
INFO:net:#### Train: [ 10/10] Step 420/4845 MSE 0.2911+-0.00000
03/01 06:20:37 AM | #### Train: [ 10/10] Step 450/4845 MSE 0.3015+-0.00000
INFO:net:#### Train: [ 10/10] Step 450/4845 MSE 0.3015+-0.00000
03/01 06:20:37 AM | #### Train: [ 10/10] Step 480/4845 MSE 0.3028+-0.00000
INFO:net:#### Train: [ 10/10] Step 480/4845 MSE 0.3028+-0.00000
03/01 06:20:37 AM | #### Train: [ 10/10] Step 510/4845 MSE 0.2917+-0.00000
INFO:net:#### Train: [ 10/10] Step 510/4845 MSE 0.2917+-0.00000
03/01 06:20:37 AM | #### Train: [ 10/10] Step 540/4845 MSE 0.2814+-0.00000
INFO:net:#### Train: [ 10/10] Step 540/4845 MSE 0.2814+-0.00000
03/01 06:20:37 AM | #### Train: [ 10/10] Step 570/4845 MSE 0.2747+-0.00000
INFO:net:#### Train: [ 10/10] Step 570/4845 MSE 0.2747+-0.00000
03/01 06:20:37 AM | #### Train: [ 10/10] Step 600/4845 MSE 0.2656+-0.00000
INFO:net:#### Train: [ 10/1

03/01 06:20:40 AM | #### Train: [ 10/10] Step 2160/4845 MSE 0.1926+-0.00000
INFO:net:#### Train: [ 10/10] Step 2160/4845 MSE 0.1926+-0.00000
03/01 06:20:40 AM | #### Train: [ 10/10] Step 2190/4845 MSE 0.1944+-0.00000
INFO:net:#### Train: [ 10/10] Step 2190/4845 MSE 0.1944+-0.00000
03/01 06:20:40 AM | #### Train: [ 10/10] Step 2220/4845 MSE 0.1936+-0.00000
INFO:net:#### Train: [ 10/10] Step 2220/4845 MSE 0.1936+-0.00000
03/01 06:20:40 AM | #### Train: [ 10/10] Step 2250/4845 MSE 0.1972+-0.00000
INFO:net:#### Train: [ 10/10] Step 2250/4845 MSE 0.1972+-0.00000
03/01 06:20:40 AM | #### Train: [ 10/10] Step 2280/4845 MSE 0.2040+-0.00000
INFO:net:#### Train: [ 10/10] Step 2280/4845 MSE 0.2040+-0.00000
03/01 06:20:41 AM | #### Train: [ 10/10] Step 2310/4845 MSE 0.2077+-0.00000
INFO:net:#### Train: [ 10/10] Step 2310/4845 MSE 0.2077+-0.00000
03/01 06:20:41 AM | #### Train: [ 10/10] Step 2340/4845 MSE 0.2113+-0.00000
INFO:net:#### Train: [ 10/10] Step 2340/4845 MSE 0.2113+-0.00000
03/01 06:20:4

INFO:net:#### Train: [ 10/10] Step 3900/4845 MSE 0.2553+-0.00000
03/01 06:20:44 AM | #### Train: [ 10/10] Step 3930/4845 MSE 0.2549+-0.00000
INFO:net:#### Train: [ 10/10] Step 3930/4845 MSE 0.2549+-0.00000
03/01 06:20:44 AM | #### Train: [ 10/10] Step 3960/4845 MSE 0.2544+-0.00000
INFO:net:#### Train: [ 10/10] Step 3960/4845 MSE 0.2544+-0.00000
03/01 06:20:44 AM | #### Train: [ 10/10] Step 3990/4845 MSE 0.2537+-0.00000
INFO:net:#### Train: [ 10/10] Step 3990/4845 MSE 0.2537+-0.00000
03/01 06:20:44 AM | #### Train: [ 10/10] Step 4020/4845 MSE 0.2535+-0.00000
INFO:net:#### Train: [ 10/10] Step 4020/4845 MSE 0.2535+-0.00000
03/01 06:20:44 AM | #### Train: [ 10/10] Step 4050/4845 MSE 0.2541+-0.00000
INFO:net:#### Train: [ 10/10] Step 4050/4845 MSE 0.2541+-0.00000
03/01 06:20:44 AM | #### Train: [ 10/10] Step 4080/4845 MSE 0.2545+-0.00000
INFO:net:#### Train: [ 10/10] Step 4080/4845 MSE 0.2545+-0.00000
03/01 06:20:44 AM | #### Train: [ 10/10] Step 4110/4845 MSE 0.2538+-0.00000
INFO:net:####

INFO:net:#### Test: [ 10/10] Step 780/1211 MSE 0.7113+-0.03429
03/01 06:20:57 AM | #### Test: [ 10/10] Step 810/1211 MSE 0.7072+-0.03436
INFO:net:#### Test: [ 10/10] Step 810/1211 MSE 0.7072+-0.03436
03/01 06:20:57 AM | #### Test: [ 10/10] Step 840/1211 MSE 0.6977+-0.03435
INFO:net:#### Test: [ 10/10] Step 840/1211 MSE 0.6977+-0.03435
03/01 06:20:58 AM | #### Test: [ 10/10] Step 870/1211 MSE 0.6859+-0.03380
INFO:net:#### Test: [ 10/10] Step 870/1211 MSE 0.6859+-0.03380
03/01 06:20:58 AM | #### Test: [ 10/10] Step 900/1211 MSE 0.7153+-0.03388
INFO:net:#### Test: [ 10/10] Step 900/1211 MSE 0.7153+-0.03388
03/01 06:20:59 AM | #### Test: [ 10/10] Step 930/1211 MSE 0.7116+-0.03379
INFO:net:#### Test: [ 10/10] Step 930/1211 MSE 0.7116+-0.03379
03/01 06:20:59 AM | #### Test: [ 10/10] Step 960/1211 MSE 0.7000+-0.03340
INFO:net:#### Test: [ 10/10] Step 960/1211 MSE 0.7000+-0.03340
03/01 06:20:59 AM | #### Test: [ 10/10] Step 990/1211 MSE 0.6951+-0.03318
INFO:net:#### Test: [ 10/10] Step 990/121

TypeError: cannot unpack non-iterable Network object